## 用keras实现卷积模型的API主要有：
* Sequential API
* Functional API

### 0. 输出环境信息

In [1]:
import platform
import tensorflow
import keras

print('Plateform: {}'.format(platform.platform()))
print('Tensorflow version: {}'.format(tensorflow.__version__))
print('Keras version: {}'.format(keras.__version__))

/home/yinyunjie/anaconda3/envs/keras/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Plateform: Linux-4.13.0-37-generic-x86_64-with-debian-stretch-sid
Tensorflow version: 1.7.0
Keras version: 2.1.5


Using TensorFlow backend.


## 1. 序贯模型（Sequential Models)

### 1.1 构建网络

In [ ]:
from keras.models import Sequential
from keras.layers import Activation, BatchNormalization, Conv2D, Dense 
from keras.layers import Dropout, Flatten, MaxPooling2D
from keras.utils import plot_model


# 初始化序贯模型对象
model = Sequential()

model.add(Conv2D(32, (3, 3), padding='same', activation='relu',
                 input_shape=(150, 150, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(512, activation='relu'))

# 模型结构
model.summary()

# 把模型结构保存为图片
plot_model(model, to_file='model.png')

### 1.2 编译

In [ ]:
from keras import optimizers

# 编译
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['acc'])

### 1.3 训练

#### 1.3.1 Data Generator

两类图片的组织结构：

train_dir
   |---cat
   |    |---0.jpg
   |    |---1.jpg
   |    |.
   |---dog
   |    |---0.jpg
   |    |---1.jpg
        |.

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen  = ImageDataGenerator(rescale=1./255)

train_dir = '.....'
validation_dir = '......'

# 直接从图片所在的目录中读取数据
train_generator = train_datagen.flow_from_directory(
    train_dir,  #图片所在目录
    target_size=(150, 150),  #图片被处理为（150, 150）
    batch_size=20,
    class_mode='binary' # 分类类型
)

validation_generator = test_datagen.flow_from_directory(
    validation_dir,
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary'
)

In [ ]:
history = model.fit_generator(
    train_generator,
    steps_per_epoch=100, # batch_size * steps_per_epoch = train number
    epochs=20,
    validation_data=validation_generator,
    validation_steps=50
)

## 2. Functional API

### 2.1 构建网络

In [ ]:
from keras.models import Model
from keras.layers import Input, Dense
from keras.utils import plot_model

input = Input(shape=(784, ), name='input')

x = Dense(512, activation='relu', name='hidden1')(input)
x1 = Dense(216, activation='relu', name='hidden2')(x)
x2 = Dense(128, activation='relu', name='hidden3')(x1)
output = Dense(10, activation='softmax', name='output')(x2)

model = Model(inputs=input, outputs=output)

# 打印网络结构
model.summary()

# plot model
plot_model(model, to_file='model_structure.png')